In [ ]:
!pip install transformers
!pip install tokenizers
'''
keras와 별도의 transformer, tokenizers 라이브러리 사용

https://pypi.org/project/transformers/#description

'''

참고자료
- https://github.com/oliverguhr/transformer-time-series-prediction  
- https://dacon.io/competitions/official/235689/codeshare/2399?dtype=recent

import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers import 